# Derivatives of Types
###### (http://conal.net/blog/posts/differentiation-of-higher-order-types)
The derivative of a type if the type of it's one-hole context. Let's begin with the standard set of Functor combinators

In [30]:
:ext TypeOperators

data Void a
data Unit a = Unit
data Const x a = Const x
data Id a = Id a
data (f :+: g) a = LeftC (f a) | RightC (g a)
data (f :*: g) a = f a :*: g a
data (f :. g) a = O (f (g a))

We use a type class to define the derivates of these functors and how to fill the one-hole context to retrieve the full shape.

In [33]:
:ext TypeFamilies
:ext LambdaCase
:ext EmptyCase

class OneHole (f :: * -> *) where
  type Deriv f :: * -> *
  fillHole :: Deriv f a -> a -> f a
  
instance OneHole Unit where
  type Deriv Unit = Void
  fillHole = \case {}
  
instance OneHole (Const x) where
  type Deriv (Const x) = Void
  fillHole = \case {}
  
instance OneHole Id where
  type Deriv Id = Unit
  fillHole _ = Id
  
instance (OneHole f, OneHole g) => OneHole (f :+: g) where
  type Deriv (f :+: g) = Deriv f :+: Deriv g
  fillHole (LeftC df) = LeftC . fillHole df
  fillHole (RightC dg) = RightC . fillHole dg
  
instance (OneHole f, OneHole g) => OneHole (f :*: g) where
  type Deriv (f :*: g) = (Deriv f :*: g) :+: (f :*: Deriv g)
  fillHole (LeftC (df :*: g)) a = fillHole df a :*: g
  fillHole (RightC (f :*: dg)) a = f :*: fillHole dg a
  
instance (OneHole f, OneHole g) => OneHole (f :. g) where
  type Deriv (f :. g) = (Deriv f :. g) :*: Deriv g
  fillHole (O dfg :*: dg) = O . fillHole dfg . fillHole dg